## Улучшение модели

Испоьзуя данные из предыдущего файла попробуем ограничить количество признаков и сравнить качество предсказания для модели с ограниченным количеством признаков.  
Для этого вначале используем обычный препроцессинг, а затем обрежем датасет с учетом опрделенной мажности признаков.

In [1]:
import numpy as np
import pandas as pd

In [2]:
pd.options.mode.chained_assignment = None

In [3]:
import pickle

In [4]:
import catboost

In [5]:
from catboost import MetricVisualizer

In [17]:
from ipynb.fs.full import process_functions as func

In [7]:
import importlib

In [18]:
importlib.reload(func)

<module 'ipynb.fs.full.process_functions' (D:\Learning\GeekBrains\_Final Project\process_functions.ipynb)>

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
pd.set_option('display.max_columns', None)

In [41]:
from catboost import cv

### Модель с ограниченным количеством признаков

Загрузим и проанализируем важность признаков для последующего удаления малозначимых.

In [185]:
importances = pd.read_csv('feature_importances.csv')

In [186]:
importances.head()

,features,importance
0,full_sq,40.565687
1,life_sq,0.757903
2,floor,1.038487
3,material,0.284146
4,build_year,1.832796


In [187]:
importances = importances.sort_values(by=['importance'], ascending=False)

In [188]:
importances.shape

(288, 2)

In [189]:
importances.head(10)

,features,importance
0,full_sq,40.565687
7,state,2.486626
6,kitch_sq,1.836963
4,build_year,1.832796
5,num_room,1.714835
281,cafe_count_5000_price_high,1.621309
227,cafe_sum_2000_max_price_avg,1.595274
225,cafe_count_2000,1.390217
105,ttk_km,1.192754
233,cafe_count_2000_price_2500,1.160971


На основании выведенного фрагмента датафрэйма четко видно, что наиболее значительное влияние имеет площадь жилья (full_sq). 
Остальные факторы намного менее значимы.
Поставим условную границу для отсечения малозначимых признаков в виде 1.0

In [190]:
importances = importances[importances['importance'] > 1.0]

In [191]:
importances.shape

(11, 2)

In [192]:
features_list = importances['features'].to_list()

In [193]:
importances.head(12)

,features,importance
0,full_sq,40.565687
7,state,2.486626
6,kitch_sq,1.836963
4,build_year,1.832796
5,num_room,1.714835
281,cafe_count_5000_price_high,1.621309
227,cafe_sum_2000_max_price_avg,1.595274
225,cafe_count_2000,1.390217
105,ttk_km,1.192754
233,cafe_count_2000_price_2500,1.160971


#### Обработка датасета с учетом важности признаков.

Загрузим еще раз исходный датасет и повторим препроцессинг обрезав малозначимые признаки

In [183]:
df = pd.read_csv('project_data/attempt_4/train.csv')

In [184]:
df.head()

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price_doc
0,1,2011-08-20,43,27.0,4.0,NaN,NaN,NaN,NaN,NaN,...,9,4,0,13,22,1,0,52,4,5850000
1,2,2011-08-23,34,19.0,3.0,NaN,NaN,NaN,NaN,NaN,...,15,3,0,15,29,1,10,66,14,6000000
2,3,2011-08-27,43,29.0,2.0,NaN,NaN,NaN,NaN,NaN,...,10,3,0,11,27,0,4,67,10,5700000
3,4,2011-09-01,89,50.0,9.0,NaN,NaN,NaN,NaN,NaN,...,11,2,1,4,4,0,0,26,3,13100000
4,5,2011-09-05,77,77.0,4.0,NaN,NaN,NaN,NaN,NaN,...,319,108,17,135,236,2,91,195,14,16331452


In [146]:
with open('mean_values.pickle', 'rb') as f:
            means_dict = pickle.load(f)

In [147]:
 means_dict

{'full_sq': 54.11184513245251,
 'life_sq': 35.99719165274595,
 'kitch_sq': 6.2605214278464585,
 'floor': 8.0,
 'num_room': 2.0}

In [148]:
features_list

['full_sq',
 'state',
 'kitch_sq',
 'build_year',
 'num_room',
 'cafe_count_5000_price_high',
 'cafe_sum_2000_max_price_avg',
 'cafe_count_2000',
 'ttk_km',
 'cafe_count_2000_price_2500',
 'floor']

In [194]:
features_list.append('timestamp')

In [195]:
features_list.append('price_doc')

In [196]:
df = df[features_list]

In [151]:
df.head()

,full_sq,state,kitch_sq,build_year,num_room,cafe_count_5000_price_high,cafe_sum_2000_max_price_avg,cafe_count_2000,ttk_km,cafe_count_2000_price_2500,floor,timestamp
0,43,NaN,NaN,NaN,NaN,0,1042.86,36,10.918587,2,4.0,2011-08-20
1,34,NaN,NaN,NaN,NaN,0,1190.48,21,3.103996,2,3.0,2011-08-23
2,43,NaN,NaN,NaN,NaN,0,977.27,24,2.927487,1,2.0,2011-08-27
3,89,NaN,NaN,NaN,NaN,1,1120.00,25,14.606501,1,9.0,2011-09-01
4,77,NaN,NaN,NaN,NaN,17,1269.23,483,1.721834,50,4.0,2011-09-05


Применим функции к датасету еще раз.  
Так как количество признаков ограничено, группировка будет осуществляться по другим принципам.

In [152]:
def num_values_process(df, means):
    """data transforming and outliers removing"""
    
    #exchange array values to single number
    indices_room = df[df['num_room'].apply(lambda x: isinstance(x, (np.ndarray)))]['num_room'].index
    for index_val in indices_room:
        df['num_room'].iloc[index_val] = round(sum(df['num_room'].iloc[index_val]/len(df['num_room'].iloc[index_val])),0)
    
    # trim too large values
    df = df.loc[(df['full_sq'] < 1000)
           & (df['floor'] < 50)
           & (df['num_room'] < 12)
           & (df['kitch_sq'] < 100)]
    
    #full square   
    df.loc[df['full_sq'] < 14, 'full_sq'] = means['full_sq']
    
    #floor
    df.loc[df['floor'] < 1, 'floor'] = means['floor']    
    
    # num room
    df.loc[df['num_room'] < 1, 'num_room'] = means['num_room']
    
    #kitchen square
    df.loc[df['kitch_sq'] < 2, 'kitch_sq'] = means['kitch_sq']
    
    return df

In [197]:
indices_room = df[df['num_room'].apply(lambda x: isinstance(x, (np.ndarray)))]['num_room'].index
for index_val in indices_room:
    df['num_room'].iloc[index_val] = round(sum(df['num_room'].iloc[index_val]/len(df['num_room'].iloc[index_val])),0)

In [198]:
indices_floor = df[df['floor'].apply(lambda x: isinstance(x, (np.ndarray)))]['floor'].index
for index_val in indices_floor:
    df['floor'].iloc[index_val] = round(sum(df['floor'].iloc[index_val]/len(df['floor'].iloc[index_val])),0)

In [199]:
indices_state = df[df['state'].apply(lambda x: isinstance(x, (np.ndarray)))]['state'].index
for index_val in indices_state:
    df['state'].iloc[index_val] = round(sum(df['state'].iloc[index_val]/len(df['state'].iloc[index_val])),0)

In [200]:
df=num_values_process(df, means_dict)

In [201]:
features_list

['full_sq',
 'state',
 'kitch_sq',
 'build_year',
 'num_room',
 'cafe_count_5000_price_high',
 'cafe_sum_2000_max_price_avg',
 'cafe_count_2000',
 'ttk_km',
 'cafe_count_2000_price_2500',
 'floor',
 'timestamp',
 'price_doc']

In [202]:
mode_state = df['state'].mode().to_list()[0]
mode_state

2.0

In [203]:
df['state'] = df['state'].fillna(mode_state)

In [204]:
df.head()

,full_sq,state,kitch_sq,build_year,num_room,cafe_count_5000_price_high,cafe_sum_2000_max_price_avg,cafe_count_2000,ttk_km,cafe_count_2000_price_2500,floor,timestamp,price_doc
7672,73.000000,2.0,11.000000,NaN,2.0,0,1166.67,3,10.868389,0,17.0,2013-04-27,10100000
8056,54.111845,3.0,12.000000,1907.0,1.0,26,1503.45,474,2.301037,69,2.0,2013-05-21,2750000
8111,85.000000,2.0,6.260521,NaN,3.0,0,1500.00,3,17.691722,1,13.0,2013-05-23,7574496
8135,53.000000,3.0,8.000000,1980.0,2.0,0,1141.03,41,11.018216,2,10.0,2013-05-25,9000000
8144,41.000000,2.0,6.260521,NaN,1.0,0,1500.00,2,20.604213,0,13.0,2013-05-27,4457400


In [205]:
means_by_state = dict()

In [206]:
for column in features_list[:-2]:
    mean_series = df.groupby('state')[column].agg(pd.Series.mode)
    means_by_state[column] = mean_series

In [207]:
for key in means_by_state:
        values = means_by_state[key]
        df[key] = df[key].fillna(df['state'].apply(lambda x: values.get(x)))  

In [208]:
df.shape

(20887, 13)

In [209]:
def year_population(value):
    if value is None:
        return None
    elif value < 22:
        return value + 2000
    elif value//1000 == 0 and value//100 == 0 and value > 22:
        return value + 1900
    elif value//1000 == 0 and value // 100 > 0:        
        return None  
    else:
        return value  

In [210]:
df['build_year'] = df['build_year'].apply(lambda x: year_population(x)) 

In [211]:
df.isna().sum()

full_sq                        0
state                          0
kitch_sq                       0
build_year                     1
num_room                       0
cafe_count_5000_price_high     0
cafe_sum_2000_max_price_avg    0
cafe_count_2000                0
ttk_km                         0
cafe_count_2000_price_2500     0
floor                          0
timestamp                      0
price_doc                      0
dtype: int64

In [212]:
build_year_mode = df['build_year'].mode().to_list()[0]
build_year_mode

1970.0

In [213]:
df['build_year']=df['build_year'].fillna(build_year_mode)

In [214]:
df.isna().sum()

full_sq                        0
state                          0
kitch_sq                       0
build_year                     0
num_room                       0
cafe_count_5000_price_high     0
cafe_sum_2000_max_price_avg    0
cafe_count_2000                0
ttk_km                         0
cafe_count_2000_price_2500     0
floor                          0
timestamp                      0
price_doc                      0
dtype: int64

In [215]:
for column in features_list[:-2]:
    if column in ['build_year', 'state', 'num_room', 'cafe_count_5000_price_high', 
                  'cafe_count_2000', 'floor', 'cafe_count_2000_price_2500']:
        df[column] = df[column].astype('int32')
    else:
        df[column] = df[column].astype('float')

In [216]:
df.head()

,full_sq,state,kitch_sq,build_year,num_room,cafe_count_5000_price_high,cafe_sum_2000_max_price_avg,cafe_count_2000,ttk_km,cafe_count_2000_price_2500,floor,timestamp,price_doc
7672,73.000000,2,11.000000,1970,2,0,1166.67,3,10.868389,0,17,2013-04-27,10100000
8056,54.111845,3,12.000000,1907,1,26,1503.45,474,2.301037,69,2,2013-05-21,2750000
8111,85.000000,2,6.260521,1970,3,0,1500.00,3,17.691722,1,13,2013-05-23,7574496
8135,53.000000,3,8.000000,1980,2,0,1141.03,41,11.018216,2,10,2013-05-25,9000000
8144,41.000000,2,6.260521,1970,1,0,1500.00,2,20.604213,0,13,2013-05-27,4457400


### Обучение и проверка модели без учета макроэкономических показателей

In [218]:
y = df['price_doc']

In [220]:
y.head()

7672    10100000
8056     2750000
8111     7574496
8135     9000000
8144     4457400
Name: price_doc, dtype: int64

In [219]:
X = df[features_list[:-2]]

In [221]:
X.head()

,full_sq,state,kitch_sq,build_year,num_room,cafe_count_5000_price_high,cafe_sum_2000_max_price_avg,cafe_count_2000,ttk_km,cafe_count_2000_price_2500,floor
7672,73.000000,2,11.000000,1970,2,0,1166.67,3,10.868389,0,17
8056,54.111845,3,12.000000,1907,1,26,1503.45,474,2.301037,69,2
8111,85.000000,2,6.260521,1970,3,0,1500.00,3,17.691722,1,13
8135,53.000000,3,8.000000,1980,2,0,1141.03,41,11.018216,2,10
8144,41.000000,2,6.260521,1970,1,0,1500.00,2,20.604213,0,13


In [225]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.33, random_state = 33)

In [226]:
cat_features = ['build_year', 'state', 'num_room', 'cafe_count_5000_price_high', 
                  'cafe_count_2000', 'floor', 'cafe_count_2000_price_2500']

In [227]:
model_cb_best = catboost.CatBoostRegressor(depth = 8,
                                           iterations = 100,
                                           learning_rate = 0.1,
                                           l2_leaf_reg = 1,
                                           cat_features = cat_features)

In [229]:
model_cb_best.fit(X_train, 
             y_train, 
             eval_set = (X_valid, y_valid),
             verbose=False, 
             plot = True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Используем тестовый датасет для оценки изменеий

In [230]:
valid_df = pd.read_csv('project_data/attempt_4/test.csv')

In [236]:
X_test = func.process(valid_df, 'test')

In [237]:
X_test = X_test[features_list[:-2]]

In [238]:
y_test = model_cb_best.predict(X_test)

In [239]:
result = pd.DataFrame(valid_df['id'])

In [240]:
result['price_doc'] = y_test.tolist()

In [241]:
result = result.set_index('id')

In [242]:
result.to_csv('submission.csv')

#### Результат
Объем датасета значительно уменьшился, но точность при этом стала хуже.

### Обучение и проверка модели с учетом макроэкономических показателей

Загрузим еще раз датасет 

In [11]:
df_macro = pd.read_csv('project_data/attempt_4/train.csv')

In [12]:
df_macro.head()

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,product_type,sub_area,area_m,raion_popul,green_zone_part,indust_part,children_preschool,preschool_quota,preschool_education_centers_raion,children_school,school_quota,school_education_centers_raion,school_education_centers_top_20_raion,hospital_beds_raion,healthcare_centers_raion,university_top_20_raion,sport_objects_raion,additional_education_raion,culture_objects_top_25,culture_objects_top_25_raion,shopping_centers_raion,office_raion,thermal_power_plant_raion,incineration_raion,oil_chemistry_raion,radiation_raion,railroad_terminal_raion,big_market_raion,nuclear_reactor_raion,detention_facility_raion,full_all,male_f,female_f,young_all,young_male,young_female,work_all,work_male,work_female,ekder_all,ekder_male,ekder_female,0_6_all,0_6_male,0_6_female,7_14_all,7_14_male,7_14_female,0_17_all,0_17_male,0_17_female,16_29_all,16_29_male,16_29_female,0_13_all,0_13_male,0_13_female,raion_build_count_with_material_info,build_count_block,build_count_wood,build_count_frame,build_count_brick,build_count_monolith,build_count_panel,build_count_foam,build_count_slag,build_count_mix,raion_build_count_with_builddate_info,build_count_before_1920,build_count_1921-1945,build_count_1946-1970,build_count_1971-1995,build_count_after_1995,ID_metro,metro_min_avto,metro_km_avto,metro_min_walk,metro_km_walk,kindergarten_km,school_km,park_km,green_zone_km,industrial_km,water_treatment_km,cemetery_km,incineration_km,railroad_station_walk_km,railroad_station_walk_min,ID_railroad_station_walk,railroad_station_avto_km,railroad_station_avto_min,ID_railroad_station_avto,public_transport_station_km,public_transport_station_min_walk,water_km,water_1line,mkad_km,ttk_km,sadovoe_km,bulvar_ring_km,kremlin_km,big_road1_km,ID_big_road1,big_road1_1line,big_road2_km,ID_big_road2,railroad_km,railroad_1line,zd_vokzaly_avto_km,ID_railroad_terminal,bus_terminal_avto_km,ID_bus_terminal,oil_chemistry_km,nuclear_reactor_km,radiation_km,power_transmission_line_km,thermal_power_plant_km,ts_km,big_market_km,market_shop_km,fitness_km,swim_pool_km,ice_rink_km,stadium_km,basketball_km,hospice_morgue_km,detention_facility_km,public_healthcare_km,university_km,workplaces_km,shopping_centers_km,office_km,additional_education_km,preschool_km,big_church_km,church_synagogue_km,mosque_km,theater_km,museum_km,exhibition_km,catering_km,ecology,green_part_500,prom_part_500,office_count_500,office_sqm_500,trc_count_500,trc_sqm_500,cafe_count_500,cafe_sum_500_min_price_avg,cafe_sum_500_max_price_avg,cafe_avg_price_500,cafe_count_500_na_price,cafe_count_500_price_500,cafe_count_500_price_1000,cafe_count_500_price_1500,cafe_count_500_price_2500,cafe_count_500_price_4000,cafe_count_500_price_high,big_church_count_500,church_count_500,mosque_count_500,leisure_count_500,sport_count_500,market_count_500,green_part_1000,prom_part_1000,office_count_1000,office_sqm_1000,trc_count_1000,trc_sqm_1000,cafe_count_1000,cafe_sum_1000_min_price_avg,cafe_sum_1000_max_price_avg,cafe_avg_price_1000,cafe_count_1000_na_price,cafe_count_1000_price_500,cafe_count_1000_price_1000,cafe_count_1000_price_1500,cafe_count_1000_price_2500,cafe_count_1000_price_4000,cafe_count_1000_price_high,big_church_count_1000,church_count_1000,mosque_count_1000,leisure_count_1000,sport_count_1000,market_count_1000,green_part_1500,prom_part_1500,office_count_1500,office_sqm_1500,trc_count_1500,trc_sqm_1500,cafe_count_1500,cafe_sum_1500_min_price_avg,cafe_sum_1500_max_price_avg,cafe_avg_price_1500,cafe_count_1500_na_price,cafe_count_1500_price_500,cafe_count_1500_price_1000,cafe_count_1500_price_1500,cafe_count_1500_price_2500,cafe_count_1500_price_4000,cafe_count_1500_price_high,big_church_count_1500,church_count_1500,mosque_count_1500,leisure_count_1500,sport_count_1500,market_count_1500,green_part_2000,prom_part_2000,office_count_2000,office_sqm_2000,trc_count_2000,trc_sqm_2000,cafe_count_2000,cafe_sum_2000_min_price_avg,cafe_sum_2000_max_price_av

In [13]:
macro = pd.read_pickle('macroeconomic.pkl')

In [14]:
macro.head()

,timestamp,gdp_quart,gdp_quart_growth,balance_trade,balance_trade_growth,net_capital_export,gdp_annual_growth,average_provision_of_build_contract,average_provision_of_build_contract_moscow,rts,micex,micex_rgbi_tr,deposits_growth,deposits_rate,mortgage_value,mortgage_growth,mortgage_rate,income_per_cap,real_dispos_income_per_cap_growth,salary_growth,retail_trade_turnover_growth,unemployment,profitable_enterpr_share,unprofitable_enterpr_share,overdue_wages_per_cap,fin_res_per_cap,marriages_per_1000_cap,divorce_rate,invest_fixed_assets_phys,pop_natural_increase,childbirth,perinatal_mort_per_1000_cap,incidence_population,rent_price_4+room_bus,rent_price_3room_bus,rent_price_2room_bus,rent_price_1room_bus,rent_price_3room_eco,rent_price_2room_eco,rent_price_1room_eco,load_of_teachers_preschool_per_teacher,students_state_oneshift,provision_nurse,load_on_doctors,hospital_beds_available_per_cap,hospital_bed_occupancy_per_year,seats_theather_rfmin_per_100000_cap,population_reg_sports_share,apartment_fund_sqm
0,2010-01-01,9995.8,4.1,18.114,14.1,-0.136524,-0.078209,5.0,6.02,1553.06,1444.78,130.32,0.002354,4.5,142968,1.550459,13.72,30789.2,0.038,0.168917,106.8,0.018,0.716,0.284,12865.0,233.351529,8.0,3.9,95.8,-0.3,10.7,5.87,696.6,156.36,76.81,59.66,45.44,40.75,34.56,27.58,721.477765,89.0495,99.4,7872.848285,846.0,302.0,0.41,22.37,230310.0
1,2010-01-02,9995.8,4.1,18.114,14.1,-0.136524,-0.078209,5.0,6.02,1553.06,1444.78,130.32,0.002354,4.5,142968,1.550459,13.72,30789.2,0.038,0.168917,106.8,0.018,0.716,0.284,12865.0,233.351529,8.0,3.9,95.8,-0.3,10.7,5.87,696.6,156.36,76.81,59.66,45.44,40.75,34.56,27.58,721.477765,89.0495,99.4,7872.848285,846.0,302.0,0.41,22.37,230310.0
2,2010-01-03,9995.8,4.1,18.114,14.1,-0.136524,-0.078209,5.0,6.02,1553.06,1444.78,130.32,0.002354,4.5,142968,1.550459,13.72,30789.2,0.038,0.168917,106.8,0.018,0.716,0.284,12865.0,233.351529,8.0,3.9,95.8,-0.3,10.7,5.87,696.6,156.36,76.81,59.66,45.44,40.75,34.56,27.58,721.477765,89.0495,99.4,7872.848285,846.0,302.0,0.41,22.37,230310.0
3,2010-01-04,9995.8,4.1,18.114,14.1,-0.136524,-0.078209,5.0,6.02,1553.06,1444.78,130.32,0.002354,4.5,142968,1.550459,13.72,30789.2,0.038,0.168917,106.8,0.018,0.716,0.284,12865.0,233.351529,8.0,3.9,95.8,-0.3,10.7,5.87,696.6,156.36,76.81,59.66,45.44,40.75,34.56,27.58,721.477765,89.0495,99.4,7872.848285,846.0,302.0,0.41,22.37,230310.0
4,2010-01-05,9995.8,4.1,18.114,14.1,-0.136524,-0.078209,5.0,6.02,1553.06,1444.78,130.32,0.002354,4.5,142968,1.550459,13.72,30789.2,0.038,0.168917,106.8,0.018,0.716,0.284,12865.0,233.351529,8.0,3.9,95.8,-0.3,10.7,5.87,696.6,156.36,76.81,59.66,45.44,40.75,34.56,27.58,721.477765,89.0495,99.4,7872.848285,846.0,302.0,0.41,22.37,230310.0


Первичный датасет предполагается обработать таким же образом, как и в предыдущих вариантах.  
Но после препроцессинга,к датасету будет присоединена таблица с макроэкономическими показателями. Присоединение будет осуществляться на по дате.  
Дополнительная обработка объединенного датасета не предполагается.

In [19]:
X, y = func.process(df_macro, 'train', True)

In [20]:
X_merged = X.merge(macro, how='left', 
                   left_on='timestamp', 
                   right_on='timestamp')

In [21]:
X_merged.head()

,timestamp,full_sq,life_sq,floor,material,build_year,num_room,kitch_sq,state,product_type,sub_area,area_m,raion_popul,green_zone_part,indust_part,children_preschool,preschool_quota,preschool_education_centers_raion,children_school,school_quota,school_education_centers_raion,school_education_centers_top_20_raion,hospital_beds_raion,healthcare_centers_raion,university_top_20_raion,sport_objects_raion,additional_education_raion,culture_objects_top_25,culture_objects_top_25_raion,shopping_centers_raion,office_raion,thermal_power_plant_raion,incineration_raion,oil_chemistry_raion,radiation_raion,railroad_terminal_raion,big_market_raion,nuclear_reactor_raion,detention_facility_raion,full_all,male_f,female_f,young_all,young_male,young_female,work_all,work_male,work_female,ekder_all,ekder_male,ekder_female,0_6_all,0_6_male,0_6_female,7_14_all,7_14_male,7_14_female,0_17_all,0_17_male,0_17_female,16_29_all,16_29_male,16_29_female,0_13_all,0_13_male,0_13_female,raion_build_count_with_material_info,build_count_block,build_count_wood,build_count_frame,build_count_brick,build_count_monolith,build_count_panel,build_count_foam,build_count_slag,build_count_mix,raion_build_count_with_builddate_info,build_count_before_1920,build_count_1921-1945,build_count_1946-1970,build_count_1971-1995,build_count_after_1995,ID_metro,metro_min_avto,metro_km_avto,metro_min_walk,metro_km_walk,kindergarten_km,school_km,park_km,green_zone_km,industrial_km,water_treatment_km,cemetery_km,incineration_km,railroad_station_walk_km,railroad_station_walk_min,ID_railroad_station_walk,railroad_station_avto_km,railroad_station_avto_min,ID_railroad_station_avto,public_transport_station_km,public_transport_station_min_walk,water_km,water_1line,mkad_km,ttk_km,sadovoe_km,bulvar_ring_km,kremlin_km,big_road1_km,ID_big_road1,big_road1_1line,big_road2_km,ID_big_road2,railroad_km,railroad_1line,zd_vokzaly_avto_km,ID_railroad_terminal,bus_terminal_avto_km,ID_bus_terminal,oil_chemistry_km,nuclear_reactor_km,radiation_km,power_transmission_line_km,thermal_power_plant_km,ts_km,big_market_km,market_shop_km,fitness_km,swim_pool_km,ice_rink_km,stadium_km,basketball_km,hospice_morgue_km,detention_facility_km,public_healthcare_km,university_km,workplaces_km,shopping_centers_km,office_km,additional_education_km,preschool_km,big_church_km,church_synagogue_km,mosque_km,theater_km,museum_km,exhibition_km,catering_km,ecology,green_part_500,prom_part_500,office_count_500,office_sqm_500,trc_count_500,trc_sqm_500,cafe_count_500,cafe_sum_500_min_price_avg,cafe_sum_500_max_price_avg,cafe_avg_price_500,cafe_count_500_na_price,cafe_count_500_price_500,cafe_count_500_price_1000,cafe_count_500_price_1500,cafe_count_500_price_2500,cafe_count_500_price_4000,cafe_count_500_price_high,big_church_count_500,church_count_500,mosque_count_500,leisure_count_500,sport_count_500,market_count_500,green_part_1000,prom_part_1000,office_count_1000,office_sqm_1000,trc_count_1000,trc_sqm_1000,cafe_count_1000,cafe_sum_1000_min_price_avg,cafe_sum_1000_max_price_avg,cafe_avg_price_1000,cafe_count_1000_na_price,cafe_count_1000_price_500,cafe_count_1000_price_1000,cafe_count_1000_price_1500,cafe_count_1000_price_2500,cafe_count_1000_price_4000,cafe_count_1000_price_high,big_church_count_1000,church_count_1000,mosque_count_1000,leisure_count_1000,sport_count_1000,market_count_1000,green_part_1500,prom_part_1500,office_count_1500,office_sqm_1500,trc_count_1500,trc_sqm_1500,cafe_count_1500,cafe_sum_1500_min_price_avg,cafe_sum_1500_max_price_avg,cafe_avg_price_1500,cafe_count_1500_na_price,cafe_count_1500_price_500,cafe_count_1500_price_1000,cafe_count_1500_price_1500,cafe_count_1500_price_2500,cafe_count_1500_price_4000,cafe_count_1500_price_high,big_church_count_1500,church_count_1500,mosque_count_1500,leisure_count_1500,sport_count_1500,market_count_1500,green_part_2000,prom_part_2000,office_count_2000,office_sqm_2000,trc_count_2000,trc_sqm_2000,cafe_count_2000,cafe_sum_2000_min_price_avg,cafe_sum_2000_max_price_avg,cafe_avg_pr

In [22]:
X_merged = X_merged.drop('timestamp', axis = 1)

In [28]:
object_columns = ['product_type', 'sub_area', 'thermal_power_plant_raion', 'incineration_raion', 'culture_objects_top_25',
                 'oil_chemistry_raion', 'radiation_raion', 'railroad_terminal_raion', 'big_market_raion',
                 'nuclear_reactor_raion', 'detention_facility_raion', 'ecology', 'water_1line', 'big_road1_1line', 'railroad_1line'
                 ]

In [29]:
discrete_columns = ['floor', 'material', 'build_year', 'num_room', 'state', 'preschool_education_centers_raion',
                   'school_education_centers_raion', 'school_education_centers_top_20_raion', 'culture_objects_top_25_raion',
                   'ID_metro', 'ID_railroad_station_walk', 'ID_railroad_station_avto', 'ID_big_road1', 'ID_big_road2', 'ID_railroad_terminal',
                   'ID_bus_terminal' ]

In [30]:
cat_features = object_columns + discrete_columns

In [31]:
X_merged.shape

(30471, 336)

In [32]:
y.shape

(30471,)

Разделим датасет на тренировочную и валидационную части

In [45]:
X_train, X_valid, y_train, y_valid = train_test_split(X_merged, y, test_size = 0.33, random_state = 33)

Применим GridSeacrch для оптимизации параметров модели с учетом расширенного датасета.

In [34]:
param_grid = {'learning_rate': [0.01, 0.03],
        'depth': [6, 7, 8, 9, 10],
        'l2_leaf_reg': [1, 2, 3, 4],
         'iterations':[80,90, 100, 110]
             }

In [35]:
model_cb_merged = catboost.CatBoostRegressor(cat_features = cat_features)

In [36]:
best_model = model_cb_merged.grid_search(param_grid, 
                                       X=X_merged, 
                                       y=y, 
                                       plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 8525463.1934825	test: 8430910.0158127	best: 8430910.0158127 (0)	total: 285ms	remaining: 22.5s
1:	learn: 8454370.4209370	test: 8360610.0641703	best: 8360610.0641703 (1)	total: 370ms	remaining: 14.4s
2:	learn: 8384501.6104310	test: 8291016.1128160	best: 8291016.1128160 (2)	total: 450ms	remaining: 11.6s
3:	learn: 8317212.1452181	test: 8223744.8929423	best: 8223744.8929423 (3)	total: 540ms	remaining: 10.3s
4:	learn: 8251181.6538551	test: 8157945.6260156	best: 8157945.6260156 (4)	total: 623ms	remaining: 9.35s
5:	learn: 8185124.4981158	test: 8093156.7856254	best: 8093156.7856254 (5)	total: 695ms	remaining: 8.57s
6:	learn: 8117616.1524068	test: 8025475.4612955	best: 8025475.4612955 (6)	total: 780ms	remaining: 8.13s
7:	learn: 8051521.5734869	test: 7959579.4628818	best: 7959579.4628818 (7)	total: 866ms	remaining: 7.8s
8:	learn: 7988397.8013872	test: 7897301.0849804	best: 7897301.0849804 (8)	total: 949ms	remaining: 7.48s
9:	learn: 7923672.5156479	test: 7833020.4179125	best: 7833020.417

79:	learn: 4836484.3997210	test: 4786952.8246654	best: 4786952.8246654 (79)	total: 6.86s	remaining: 0us

bestTest = 4786952.825
bestIteration = 79

0:	loss: 4786952.8246654	best: 4786952.8246654 (0)	total: 7.37s	remaining: 19m 31s
0:	learn: 8382570.1150132	test: 8288161.7641868	best: 8288161.7641868 (0)	total: 81ms	remaining: 6.4s
1:	learn: 8174225.4091772	test: 8082122.5232344	best: 8082122.5232344 (1)	total: 152ms	remaining: 5.91s
2:	learn: 7974081.8018269	test: 7882871.2227316	best: 7882871.2227316 (2)	total: 227ms	remaining: 5.84s
3:	learn: 7781287.8656359	test: 7689739.0800395	best: 7689739.0800395 (3)	total: 307ms	remaining: 5.84s
4:	learn: 7600814.3277541	test: 7510053.9549941	best: 7510053.9549941 (4)	total: 389ms	remaining: 5.84s
5:	learn: 7420975.4054717	test: 7332923.3465667	best: 7332923.3465667 (5)	total: 466ms	remaining: 5.75s
6:	learn: 7249662.4252317	test: 7161652.1432506	best: 7161652.1432506 (6)	total: 538ms	remaining: 5.61s
7:	learn: 7081413.2058057	test: 6997305.282

77:	learn: 2966851.4618850	test: 3064952.9795074	best: 3064952.9795074 (77)	total: 6.22s	remaining: 160ms
78:	learn: 2955426.6453507	test: 3056430.2866355	best: 3056430.2866355 (78)	total: 6.29s	remaining: 79.7ms
79:	learn: 2945502.5647750	test: 3047076.2026769	best: 3047076.2026769 (79)	total: 6.37s	remaining: 0us

bestTest = 3047076.203
bestIteration = 79

1:	loss: 3047076.2026769	best: 3047076.2026769 (1)	total: 13.8s	remaining: 18m 7s
0:	learn: 8525763.2036584	test: 8431193.7719156	best: 8431193.7719156 (0)	total: 86.8ms	remaining: 6.86s
1:	learn: 8454810.9674151	test: 8361039.8568066	best: 8361039.8568066 (1)	total: 157ms	remaining: 6.11s
2:	learn: 8385130.4481960	test: 8291608.4528614	best: 8291608.4528614 (2)	total: 225ms	remaining: 5.77s
3:	learn: 8318163.2547285	test: 8224634.6887730	best: 8224634.6887730 (3)	total: 310ms	remaining: 5.89s
4:	learn: 8252352.0256693	test: 8159037.1225114	best: 8159037.1225114 (4)	total: 396ms	remaining: 5.94s
5:	learn: 8186627.5430075	test: 8094

75:	learn: 4963677.8772488	test: 4908811.8386096	best: 4908811.8386096 (75)	total: 6.05s	remaining: 318ms
76:	learn: 4934891.8172078	test: 4881134.2057846	best: 4881134.2057846 (76)	total: 6.14s	remaining: 239ms
77:	learn: 4905939.5452194	test: 4853271.9803774	best: 4853271.9803774 (77)	total: 6.21s	remaining: 159ms
78:	learn: 4876825.5266756	test: 4825422.8450798	best: 4825422.8450798 (78)	total: 6.31s	remaining: 79.8ms
79:	learn: 4847780.5790221	test: 4797548.1102177	best: 4797548.1102177 (79)	total: 6.38s	remaining: 0us

bestTest = 4797548.11
bestIteration = 79

2:	loss: 4797548.1102177	best: 3047076.2026769 (1)	total: 20.2s	remaining: 17m 36s
0:	learn: 8383470.8168545	test: 8289014.7886446	best: 8289014.7886446 (0)	total: 87ms	remaining: 6.87s
1:	learn: 8175544.1085460	test: 8083420.4962046	best: 8083420.4962046 (1)	total: 164ms	remaining: 6.38s
2:	learn: 7975917.0944473	test: 7884614.7722793	best: 7884614.7722793 (2)	total: 232ms	remaining: 5.95s
3:	learn: 7784474.1346363	test: 76

...

17:	learn: 5647471.0987972	test: 5673422.3144941	best: 5673422.3144941 (17)	total: 10.9s	remaining: 55.6s
18:	learn: 5530903.7489746	test: 5559211.1558167	best: 5559211.1558167 (18)	total: 11.5s	remaining: 55s
19:	learn: 5416543.0772100	test: 5447871.0059827	best: 5447871.0059827 (19)	total: 12.1s	remaining: 54.4s
20:	learn: 5307616.2598731	test: 5342967.9648765	best: 5342967.9648765 (20)	total: 12.7s	remaining: 53.8s
21:	learn: 5202300.2684058	test: 5241964.6879717	best: 5241964.6879717 (21)	total: 13.3s	remaining: 53.2s
22:	learn: 5099120.9732151	test: 5140878.8148235	best: 5140878.8148235 (22)	total: 13.9s	remaining: 52.6s
23:	learn: 5000819.7836159	test: 5046695.5429753	best: 5046695.5429753 (23)	total: 14.5s	remaining: 52s
24:	learn: 4902416.0785099	test: 4952108.1465954	best: 4952108.1465954 (24)	total: 15.1s	remaining: 51.3s
25:	learn: 4808188.0750692	test: 4862219.5994047	best: 4862219.5994047 (25)	total: 15.7s	remaining: 50.7s
26:	learn: 4718793.0343186	test: 4777230.5976916	b

95:	learn: 2482716.3130670	test: 2868549.4134995	best: 2868549.4134995 (95)	total: 1m 1s	remaining: 8.97s
96:	learn: 2473987.9143277	test: 2862680.7902108	best: 2862680.7902108 (96)	total: 1m 2s	remaining: 8.32s
97:	learn: 2465634.0378880	test: 2858066.3750781	best: 2858066.3750781 (97)	total: 1m 2s	remaining: 7.68s
98:	learn: 2456725.3472752	test: 2854165.9852208	best: 2854165.9852208 (98)	total: 1m 3s	remaining: 7.03s
99:	learn: 2448492.4862114	test: 2849792.4753731	best: 2849792.4753731 (99)	total: 1m 3s	remaining: 6.39s
100:	learn: 2442372.8968427	test: 2845573.8780293	best: 2845573.8780293 (100)	total: 1m 4s	remaining: 5.75s
101:	learn: 2436756.6944244	test: 2842941.8664550	best: 2842941.8664550 (101)	total: 1m 5s	remaining: 5.11s
102:	learn: 2429818.3888525	test: 2840603.3358148	best: 2840603.3358148 (102)	total: 1m 5s	remaining: 4.47s
103:	learn: 2422028.1441785	test: 2836929.9402614	best: 2836929.9402614 (103)	total: 1m 6s	remaining: 3.83s
104:	learn: 2410943.9983735	test: 2832

62:	learn: 2888467.9050204	test: 3386395.1524996	best: 3386395.1524996 (62)	total: 39.7s	remaining: 29.6s
63:	learn: 2866421.8821755	test: 3370985.4966366	best: 3370985.4966366 (63)	total: 40.3s	remaining: 29s
64:	learn: 2844527.5950769	test: 3354122.0773324	best: 3354122.0773324 (64)	total: 40.9s	remaining: 28.3s
65:	learn: 2822659.1690524	test: 3337520.3143462	best: 3337520.3143462 (65)	total: 41.5s	remaining: 27.7s
66:	learn: 2800697.0717163	test: 3320190.8081580	best: 3320190.8081580 (66)	total: 42.1s	remaining: 27s
67:	learn: 2780490.2820981	test: 3304843.0870516	best: 3304843.0870516 (67)	total: 42.7s	remaining: 26.4s
68:	learn: 2761530.5812770	test: 3291024.4774798	best: 3291024.4774798 (68)	total: 43.3s	remaining: 25.7s
69:	learn: 2742117.5355235	test: 3277661.4449856	best: 3277661.4449856 (69)	total: 43.9s	remaining: 25.1s
70:	learn: 2722988.7624866	test: 3264703.9894236	best: 3264703.9894236 (70)	total: 44.5s	remaining: 24.4s
71:	learn: 2704944.6157956	test: 3252620.0104967	b

29:	learn: 4488579.1438015	test: 4498486.2181598	best: 4498486.2181598 (29)	total: 18.2s	remaining: 48.5s
30:	learn: 4414609.9287184	test: 4428997.7972688	best: 4428997.7972688 (30)	total: 18.8s	remaining: 47.9s
31:	learn: 4337420.1872135	test: 4357893.2957114	best: 4357893.2957114 (31)	total: 19.4s	remaining: 47.3s
32:	learn: 4264471.9125735	test: 4288853.0682016	best: 4288853.0682016 (32)	total: 20s	remaining: 46.7s
33:	learn: 4194396.7524411	test: 4222773.9688606	best: 4222773.9688606 (33)	total: 20.6s	remaining: 46.1s
34:	learn: 4128034.9042927	test: 4159046.9637351	best: 4159046.9637351 (34)	total: 21.2s	remaining: 45.5s
35:	learn: 4066140.5220013	test: 4100894.5685625	best: 4100894.5685625 (35)	total: 21.8s	remaining: 44.8s
36:	learn: 4002299.8288315	test: 4041457.9287959	best: 4041457.9287959 (36)	total: 22.4s	remaining: 44.2s
37:	learn: 3942365.9903415	test: 3985975.8730151	best: 3985975.8730151 (37)	total: 23s	remaining: 43.6s
38:	learn: 3884864.0479759	test: 3933391.0899030	b

107:	learn: 2377611.8206961	test: 2752791.7380403	best: 2752791.7380403 (107)	total: 1m 5s	remaining: 1.21s
108:	learn: 2369666.6564378	test: 2749152.6822513	best: 2749152.6822513 (108)	total: 1m 6s	remaining: 608ms
109:	learn: 2362832.5288072	test: 2743938.4128519	best: 2743938.4128519 (109)	total: 1m 7s	remaining: 0us

bestTest = 2743938.413
bestIteration = 109



In [39]:
best_params = best_model['params']
best_params

{'depth': 10, 'l2_leaf_reg': 1, 'iterations': 110, 'learning_rate': 0.03}

Проведем обучение модели на основоании выбранных параметров.  
Дополнительно введем условие ранней остановки для исбежания переобучения.

In [48]:
model_cb_best = catboost.CatBoostRegressor(depth=10, 
                                           l2_leaf_reg=1, 
                                           iterations=110, 
                                           learning_rate=0.03, 
                                           cat_features = cat_features,
                                           early_stopping_rounds=20)

In [49]:
cb_model_with_best_params = model_cb_merged.fit(X_train, 
                                               y_train,
                                               eval_set=(X_valid, y_valid),
                                               verbose=False,
                                               plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [52]:
cb_model_with_best_params.get_feature_importance(prettified=True)

,Feature Id,Importances
0,full_sq,42.566859
1,cafe_count_5000_price_2500,2.437925
2,state,2.391561
3,cafe_count_3000_price_2500,1.799236
4,cafe_sum_2000_min_price_avg,1.245696
...,...,...
331,retail_trade_turnover_growth,0.000000
332,childbirth,0.000000
333,perinatal_mort_per_1000_cap,0.000000
334,load_of_teachers_preschool_per_teacher,0.000000


Проверим полученную модель на предсказаниях для тестового датасета.

In [53]:
valid_df = pd.read_csv('project_data/attempt_4/test.csv')

In [55]:
X_test = func.process(valid_df, 'test', True)

In [56]:
X_test_merged = X_test.merge(macro, how='left', 
                   left_on='timestamp', 
                   right_on='timestamp')

In [57]:
X_test_merged = X_test_merged.drop('timestamp', axis = 1)

In [58]:
submissions = cb_model_with_best_params.predict(X_test_merged)

In [59]:
result = pd.DataFrame(valid_df['id'])

In [60]:
result['price_doc'] = submissions.tolist()

In [61]:
result = result.set_index('id')

In [62]:
result.head()

,price_doc
id,
30474,5.703349e+06
30475,8.153889e+06
30476,5.846560e+06
30477,5.691575e+06
30478,5.253600e+06


In [63]:
result[result['price_doc'] < 0].count()

price_doc    0
dtype: int64

In [64]:
result.to_csv('submission.csv')

**Итог:**  
Значение оценки полученное на реальном датасете 0.33751.
Этот результат ниже,чем показанный без присоединения макроэкономичесиъ показателей.
Можно предположить, что при дальнейшем усовершенствовании модели, более тщательном поборе гиперпараметров и измененной предобработке признаков есть вероятность получить более высокие результаты.